# FPS Victory Bayesian Model

In this notebook, we describe a very simple Bayesian network implemented using pgmpy library, that can be used to assess victory ofan 1v1 match, given two opponents, including a penalty/bonus mechanism for situations when the two opponents’ skills are disproportionate.

# Imports

In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Model Definition

As our model, we'll use a Bayesian Network inspired by [Microsoft's TrueSkill](https://www.microsoft.com/en-us/research/project/trueskill-ranking-system/), where we'll only have discrete variables (as pgmpy doesn't properly support continuous variables at the time of writing this) with a much smaller domain, we'll also limit our model to a 1v1 situation.


We define our model by first defining the nodes:

*   **Player’s W/L ≥ 50%**: whether the player has a Win/Lose ratio in
above 50 % of matches (i.e. wins more than half of the matches).
*   **Player’s K/D ratio**: kill/death ratio of a player, telling us how
many other players a player usually kills before dying. We consider
this quantity as a discrete random variable with three possible values:
Neg for a k/d ratio < 1.0, Pos for a k/d ratio between 1.0 and 2.0,
and Double for a k/d ratio > 2.0.
*   **Player’s Skill**: this variable is used to model a player’s skill level, being conditioned on previous two variables W/L and K/D stats. In our
case, we assume three skill levels, namely: bad, average and good. The
conditioning on player’s stats changes the probability of that player of
being of a certain skill level (i.e. a player with a W/L < 50 and K/D
< 1.0 is more likely to be assigned ”Bad” skill level).
*  **Player’s Performance**: this variable is used to model the fact that
players do not always perform according to their skill in all of their
games, in fact good players can be beaten by bad players (i.e. it was
not their day, or they weren’t taking the game seriously, etc.). In our
case, the domain of this variable is Bad/Good, where of course we
model the fact that a player is more likely to perform well if his/her
skill level is higher.
*  **Victory A**: this variable, conditioned on performances of both players,
tells us whether player A wins or not.
*  **Bonus**: this variable has a domain of -1/0/1, and is used to model
the impact of a player’s win. Namely, we want to reward a lower skill
player if he/she beats a higher skill player, and penalize a high skill player if he/she loses against a lower skill player. This variable is thus
conditioned on Skill A, Skill B and Victory A variables.

Players' nodes are defined in a symmetric way, so we'll have the K/D, W/L, Skill and Performance nodes defined for player A and player B.

**Note**: Our choice of values for K/D ratio, very crudely approximates the values that are generally looked upon, by players’ communities in an average FPS
game (i.e. players with negative k/d ratio are considered bad, while
players with a ratio above 2.0 are considered expert).

Finally, we need to define connections, which we'll do as in the following image:

<img alt="Model graph" src="./figures/model_graph.png" width="400">

In [ ]:
victory_model = BayesianModel([
    ("WL_A", "Skill_A"),
    ("WL_B", "Skill_B"),
    ("KD_A", "Skill_A"),
    ("KD_B", "Skill_B"),
    ("Skill_A", "Perf_A"),
    ("Skill_B", "Perf_B"),                              
    ("Perf_A", "Victory_A"),
    ("Perf_B", "Victory_A"),
    ("Skill_A", "Bonus"),
    ("Skill_B", "Bonus"),
    ("Victory_A", "Bonus")
])

# CPT Definition

In [ ]:
# assume W/L: Neg if w/l < 50%, Pos if w/l >= 50%
cpd_WL_50_A = TabularCPD(variable='WL_A', variable_card=2, values=[[0.5], [0.5]], state_names={"WL_A": ["Neg", "Pos"]})
cpd_WL_50_B = TabularCPD(variable='WL_B', variable_card=2, values=[[0.5], [0.5]], state_names={"WL_B": ["Neg", "Pos"]})

# assume KD: Neg if kd < 1.0, Pos if 1<=kd<=2.0, Double if kd>2.0
cpd_KD_A = TabularCPD(variable='KD_A', variable_card=3, values=[[0.4], [0.4], [0.2]], state_names={'KD_A': ['Neg', 'Pos', 'Double']})
cpd_KD_B = TabularCPD(variable='KD_B', variable_card=3, values=[[0.4], [0.4], [0.2]], state_names={'KD_B': ['Neg', 'Pos', 'Double']})

cpd_Skill_A = TabularCPD(variable='Skill_A', variable_card=3, 
    values=[[0.93, 0.5, 0.1, 0.6, 0.2, 0.05],
            [0.05, 0.4, 0.6, 0.35, 0.7, 0.1],
            [0.02, 0.1, 0.3, 0.05, 0.1, 0.85]],
    evidence=['WL_A', 'KD_A'],
    evidence_card=[2, 3],
    state_names={"Skill_A": ['1', '2', '3'], 'KD_A': ['Neg', 'Pos', 'Double'], "WL_A": ["Neg", "Pos"]}
)

cpd_Skill_B = TabularCPD(variable='Skill_B', variable_card=3, 
    values=[[0.93, 0.5, 0.1, 0.6, 0.2, 0.05],
            [0.05, 0.4, 0.6, 0.35, 0.7, 0.1],
            [0.02, 0.1, 0.3, 0.05, 0.1, 0.85]],
    evidence=['WL_B', 'KD_B'],
    evidence_card=[2, 3],
    state_names={"Skill_B": ['1', '2', '3'], 'KD_B': ['Neg', 'Pos', 'Double'], "WL_B": ["Neg", "Pos"]}
)

cpd_Perf_A = TabularCPD(variable='Perf_A', variable_card=2, 
    values=[[0.6, 0.35, 0.15],
            [0.4, 0.65, 0.85]],
    evidence=['Skill_A'],
    evidence_card=[3],
    state_names = {'Perf_A': ['Bad', 'Good'], "Skill_A": ['1', '2', '3']}
)

cpd_Perf_B = TabularCPD(variable='Perf_B', variable_card=2, 
    values=[[0.6, 0.35, 0.15],
            [0.4, 0.65, 0.85]],
    evidence=['Skill_B'],
    evidence_card=[3],
    state_names = {'Perf_B': ['Bad', 'Good'], "Skill_B": ['1', '2', '3']}
)

cpd_Victory_A = TabularCPD(variable='Victory_A', variable_card=2, 
    values=[[0.5, 0.7, 0.3, 0.5],
            [0.5, 0.3, 0.7, 0.5]],
    evidence=['Perf_A', 'Perf_B'],
    evidence_card=[2, 2],
    state_names = {'Victory_A': ['No', 'Yes'], 'Perf_A': ['Bad', 'Good'], 'Perf_B': ['Bad', 'Good']}
)

cpd_Bonus = TabularCPD(variable='Bonus', variable_card=3, 
    values=[[0.05, 0.0, 0.0, 0.6, 0.05, 0.0, 0.8, 0.6, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.95, 1.0, 1.0, 0.4, 0.95, 1.0, 0.2, 0.4, 0.9, 0.5, 0.35, 0.1, 1.0, 0.5, 0.35, 1.0, 0.95, 0.8],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.65, 0.9, 0.0, 0.5, 0.65, 0.0, 0.05, 0.2]],
    evidence=['Victory_A', 'Skill_A', 'Skill_B'],
    evidence_card=[2, 3, 3],
    state_names = {'Bonus': ["-1", "0", "1"], 'Victory_A': ['No', 'Yes'], 'Skill_A': ['1', '2', '3'], 'Skill_B': ['1', '2', '3']}
)

In [ ]:
victory_model.add_cpds(cpd_WL_50_A, cpd_WL_50_B, cpd_KD_A, cpd_KD_B, cpd_Skill_A, cpd_Skill_B, cpd_Perf_A, cpd_Perf_B, cpd_Victory_A, cpd_Bonus)
victory_model.check_model()

# CPDs viewing

In [ ]:
print(cpd_Skill_A)
print(cpd_Perf_A)
print(cpd_Victory_A)
print(cpd_Bonus)

# Independences viewing

In [ ]:
print(victory_model.local_independencies('Victory_A'))
print(victory_model.local_independencies('Skill_A'))
print(victory_model.local_independencies('Perf_A'))

# Inference queries

In [ ]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(victory_model)

The distribution over Victory_A without any evidence, is in fact 50/50

In [ ]:
victory_A_dist = infer.query(['Victory_A'])
print(victory_A_dist)

If we know that KD of A is > 2.0 (i.e. he usually kills at least 2 players before dying, then his chance to win rises)

In [ ]:
print(infer.query(['Victory_A'], evidence={'KD_A': 'Double'}))

If we also add the fact that his opponent loses more than half matches, and that is bad overall

In [ ]:
print(infer.query(['Victory_A'], evidence={'KD_A': 'Double', "WL_B": 'Neg'}))
print(infer.query(['Victory_A'], evidence={'KD_A': 'Double', "WL_B": 'Neg', 'KD_B': 'Neg'}))
print(infer.query(['Victory_A'], evidence={'WL_A': 'Pos', 'KD_A': 'Double', "WL_B": 'Neg', 'KD_B': 'Neg'}))

If we know that A loses most of the games, but has double KD, the probability of winning drops

In [ ]:
print(infer.query(['Victory_A'], evidence={'WL_A': 'Neg', 'KD_A': 'Double', "WL_B": 'Neg', 'KD_B': 'Neg'}))

However, if we know that B performs poorly, then A is more likely to win

In [ ]:
print(infer.query(['Victory_A'], evidence={'WL_A': 'Neg', 'KD_A': 'Double', "WL_B": 'Neg', 'KD_B': 'Neg', 'Perf_B': "Bad"}))
# But since if we know the performance, we don't care about the stats and skill, we get the same probability,
# thus: Victory_A _|_ WL_B, KD_B | Perf_B
print(infer.query(['Victory_A'], evidence={'WL_A': 'Neg', 'KD_A': 'Double', 'Perf_B': "Bad"}))

What's the probability that A performed well if he won?

In [ ]:
print(infer.query(['Perf_A'], evidence={'Victory_A': "Yes"}))
# What's the probability that A was skilled 3 if he won?
# Surprisingly, we don't get to know much about Skill of A even if we know he won against a skilled player
print(infer.query(['Skill_A'], evidence={'Victory_A': "Yes", "Skill_B": "3"}))

However, if we know that the bonus was 0, it is most likely because he won against an oponent of same level, thus B of skill 3, thus A is more likely to be of skill 3

In [ ]:
print(infer.query(['Skill_A'], evidence={'Victory_A': "Yes", "Skill_B": "3", "Bonus": "0"}))

In [ ]:
print(infer.map_query(['Victory_A', 'Bonus'], evidence={'KD_A': "Double", "KD_B": "Pos"}))
print(infer.map_query(['Victory_A', 'Bonus'], evidence={'WL_A': "Neg", "KD_B": "Pos"}))

If a "bad" player beats a better one, then he will get bonus

In [ ]:
print(infer.map_query(['Bonus'], evidence={'Victory_A': "Yes" ,'KD_A': "Neg", "WL_A": "Neg", "KD_B": "Pos"}))